## 1. Info

In [12]:
import os 

DATAPATH = './FW&BW_Rawdata/'

In [88]:
files = os.listdir(DATAPATH + 'PD')
fnames = set()

for f in files:
    fnames.add(f[:3])
    
print('PD:', len(fnames))

PD: 81


In [89]:
files = os.listdir(DATAPATH + 'Controls')
fnames = set()

for f in files:
    fnames.add(f[:3])
    
print('Controls:', len(fnames))

Controls: 15


## 2. Prep

In [60]:
import pandas as pd

RAWDATAPATH = './FW&BW_Rawdata/PD'
DATASETPATH = './dataset/'

if os.path.exists(DATASETPATH) == False:
    os.mkdir(DATASETPATH)

# TARGET_FILE = 'AMJ_BW1.csv' 

In [61]:
def prepData(TARGET_PATH, ptype):
    # 1. 2줄 제외하고 읽기
    dff = pd.read_csv(TARGET_PATH, skiprows=2, encoding='utf-8')
    
    # 2. 전체 none인 컬럼 제외
    while dff[list(dff.columns)[-1]].isna().all():
        dff.drop(list(dff.columns)[-1], axis=1, inplace=True)
        
    # 3. 단위가 mm인 데이터만 추출
    target = [0, 1]
    target += list(filter(lambda x : list(dff.iloc[1] == 'mm')[x], range(len(list(dff.iloc[1])))))
    dff = dff.iloc[:, target]
    dff.drop(dff.index[1], inplace=True)
    
    # 4. 컬럼 이름 변경하기 

    pList = dff.columns
    colList = dff.iloc[0]

    NAME = None

    pNames = []
    for p in pList:
        if not p.startswith('Unnamed'):
            if NAME == None: NAME = p.split(':')[0]
            pNames.append(p.split(':')[1])
            
#     print(pNames)
#     print(colList[2:])

#     assert len(pNames) == 39 and len(colList[2:]) == 39*3

    columns = []
    columns.append(colList[0])
    columns.append(colList[1])
    for idx, col in enumerate(colList[2:]):
        i = int(idx / 3)
        eachColName = pNames[i] + '_' + col
        columns.append(eachColName)

    assert len(columns) == len(list(dff.columns))

    dff.drop(dff.index[0], inplace=True)
    dff.reset_index(drop=True, inplace=True)

    dff.columns = columns
    
    if os.path.exists(DATASETPATH + ptype) == False:
        os.mkdir(DATASETPATH + ptype)
        
    dff.to_csv(os.path.join(DATASETPATH + ptype, 'PREP_'+TARGET_FILE), encoding='utf-8')
    
#     print(dff.head())

In [63]:
RAWDATAPATH = './FW&BW_Rawdata/PD'
for TARGET_FILE in os.listdir(RAWDATAPATH):
    if TARGET_FILE.endswith('.csv'):
        prepData(os.path.join(RAWDATAPATH, TARGET_FILE), RAWDATAPATH.split('/')[-1])

In [67]:
RAWDATAPATH = './FW&BW_Rawdata/Controls'
for TARGET_FILE in os.listdir(RAWDATAPATH):
    if TARGET_FILE.endswith('.csv'):
        prepData(os.path.join(RAWDATAPATH, TARGET_FILE), RAWDATAPATH.split('/')[-1])

In [68]:
RAWDATAPATH = './FW&BW_Rawdata/Controls(추가)'
for TARGET_FILE in os.listdir(RAWDATAPATH):
    if TARGET_FILE.endswith('.csv'):
        prepData(os.path.join(RAWDATAPATH, TARGET_FILE), RAWDATAPATH.split('/')[-1])

In [100]:
files = os.listdir(DATASETPATH + 'PD')
fnames = set()

for f in files:
    if f.endswith('.csv'):
        fnames.add(f.split('_')[1])
    
print('PD:', len(fnames))

files = os.listdir(DATASETPATH + 'Controls')
fnames = set()

for f in files:
    if f.endswith('.csv'):
        fnames.add(f.split('_')[1])
    
print('Controls:', len(fnames))

PD: 83
Controls: 22
